# Notebook pour le calcul des indices NDWI, LST (MODIS)

### Initialisation contexte test appli

In [ ]:
import os

# Manually filling Config dictionnary (can be filled by reading Config_process.env) :

CONFIG = {}

CONFIG['WRK_DIR'] = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(CONFIG['WRK_DIR'])
CONFIG['WRK_DIR'] = os.path.normpath('Y:/EO4DM')
CONFIG['DATA_HISTO'] = os.path.join(CONFIG['WRK_DIR'],'DATA_HISTO')
CONFIG['ANNEX_DIR'] = os.path.join(CONFIG['WRK_DIR'],'ANNEX')

CONFIG['TERRITORY'] = 'Fiji'
CONFIG['MODE'] = 'MANUAL'                                 # 'AUTO', 'MANUAL', 'INDICES' or 'DROUGHT' mode of drought processing chain
CONFIG['PERIOD_START'] = '2000-03-03'
CONFIG['PERIOD_END'] = '2000-03-04'
CONFIG['DRIVE_FOLDER'] = 'EO4DM_EXPORT_NOTEBOOK'
CONFIG['LANDMASK_ROI'] = 0                                # if 1, preproc products are masked for pixels outside landmask roi (input shapefile to upload on gee asset)
CONFIG['ASSET_EXPORT_L'] = 0                              # if 1, landsat preproc products are exported to gee
CONFIG['ASSET_EXPORT_S2'] = 0                             # if 1, s2 preproc products are exported to gee
CONFIG['ASSET_EXPORT_MOD'] = 0                            # if 1, modis preproc products are exported to gee
CONFIG['CLEAN_GEEFOLDER'] = 0                             # if 1, gee already existing folder is cleaned (deleted, and re-created)
CONFIG['CLEAN_GEECOL'] = 0                                # if 1, gee already exported products are cleaned (deleted, and re-created)
CONFIG['CLEAN_RUNFOLDER'] = 0                         # if 1, drought already existing folder is cleaned (deleted, and re-created)

CONFIG['lon_min_modis'] = ''
CONFIG['lat_min_modis'] = ''
CONFIG['lon_max_modis'] = ''
CONFIG['lat_max_modis'] = ''
# CONFIG['lon_min_modis'] = 177.039
# CONFIG['lat_min_modis'] = -19.245
# CONFIG['lon_max_modis'] = 180.352
# CONFIG['lat_max_modis'] = -16.074
CONFIG['DROUGHT_STATS'] = 0


### Création arborescence dossiers DATA et controle ANNEX_DIR

In [ ]:
import dmpipeline.DROUGHT_Processing.DROUGHT_global_functions as dglob
dglob.prepare_DATAFolders(CONFIG)

### Test détection auto de la période à traiter (en fonction de data_histo)

In [ ]:
import dmpipeline.DROUGHT_Processing.DROUGHT_global_functions as dglob
go_process, CONFIG['PERIOD_START'], CONFIG['PERIOD_END'] = dglob.auto_processingPERIOD(CONFIG)

print(f'go_process = {go_process}')
PERIOD_START = CONFIG['PERIOD_START']
PERIOD_END = CONFIG['PERIOD_END']
print(f'PERIOD_START = {PERIOD_START}')
print(f'PERIOD_END = {PERIOD_END}')


### Test de disponibilité des produits GEE

In [ ]:
import dmpipeline.GEE_Processing.GEE_main_functions as geemain
go_modis, go_month, CONFIG['PERIOD_START'], CONFIG['PERIOD_END'], CONFIG['TERRITORY'] = geemain.check_GEEProducts_MODIS(CONFIG)

print(f'go_modis = {go_modis}')
print(f'go_month = {go_month}')
PERIOD_START = CONFIG['PERIOD_START']
PERIOD_END = CONFIG['PERIOD_END']
print(f'PERIOD_START = {PERIOD_START}')
print(f'PERIOD_END = {PERIOD_END}')

In [ ]:
import dmpipeline.DROUGHT_Processing.DROUGHT_global_functions as dglob
OUTDIR_PATHS = dglob.prepare_RUNFolders(CONFIG)

### Test auto détermination de la Bounding Box

In [ ]:
CONFIG['lon_min_modis'], CONFIG['lat_min_modis'], CONFIG['lon_max_modis'], CONFIG['lat_max_modis'], CONFIG['TRAN_OUT'] = dglob.auto_processingBBOX(CONFIG)

In [ ]:
CONFIG['lon_min_modis'], CONFIG['lat_min_modis'], CONFIG['lon_max_modis'], CONFIG['lat_max_modis'], CONFIG['TRAN_OUT'] 

In [ ]:
TRAN_OUT =  CONFIG['TRAN_OUT']

In [ ]:
import dmpipeline.GEE_Processing.GEE_main_functions as geemain
COLLECTIONS, PARAM, DICT = geemain.initialize_GEEProcessing_MODIS(CONFIG, OUTDIR_PATHS[3])

### Vérification bonne lecture GEE des collections MODIS

In [ ]:
# LSTaqua_dataset = COLLECTIONS[0]
# LSTterra_dataset = COLLECTIONS[1]
# REFLECTterra_dataset = src_colLECTIONS[2]
# new_collection = COLLECTIONS[3]

# Nb_LSTaqua = LSTaqua_dataset.size().getInfo()
# Nb_LSTterra = LSTterra_dataset.size().getInfo()
# Nb_REFLECTterra = REFLECTterra_dataset.size().getInfo()

# TERRITORY = CONFIG['TERRITORY']

# print(f' - TERRITORY : {TERRITORY}\n - PERIOD : {PERIOD_START} -> {PERIOD_END}\n - NB LST AQUA : {Nb_LSTaqua}\n - NB LST TERRA : {Nb_LSTterra}\n - NB REFLECT TERRA : {Nb_REFLECTterra}\n')
# print(f'NEW GEE Collection path:\n{new_collection}\n')

### Extrait collections (mois m et année y)

In [ ]:
# import ee
# import dmpipeline.GEE_Processing.gee_accounts as geeauth

# path2key = os.path.dirname(geeauth.__file__)

# landmask =  COLLECTIONS[4]
# (GRID_OUT, SALE_OUT, date_start, date_end) = PARAM
# (QA_dict, QAtable_filename, QAtable_columns, COMP_dict, COMPtable_filename, COMPtable_columns) = DICT

# # y=2002
# # m=7
# y = ee.Date(date_start).get('year').getInfo()
# m = ee.Date(date_start).get('month').getInfo()

# LSTaqua_month = (LSTaqua_dataset
#                 .filter(ee.Filter.calendarRange(y,y,'year'))
#                 .filter(ee.Filter.calendarRange(m,m,'month')))
# LSTterra_month = (LSTterra_dataset
#                   .filter(ee.Filter.calendarRange(y,y,'year'))
#                   .filter(ee.Filter.calendarRange(m,m,'month')))
# REFLECTterra_month = (REFLECTterra_dataset
#                       .filter(ee.Filter.calendarRange(y,y,'year'))
#                       .filter(ee.Filter.calendarRange(m,m,'month')))


### Calcul indices MODIS jour Di

In [ ]:
# # Extrait produits jour=di
# di = 1

# lst_aqua = (LSTaqua_month
#                 .filter(ee.Filter.calendarRange(di,di,'DAY_OF_MONTH'))
#                 .first())
# lst_terra = (LSTterra_month
#                 .filter(ee.Filter.calendarRange(di,di,'DAY_OF_MONTH'))
#                 .first())
# reflect = (REFLECTterra_month
#                 .filter(ee.Filter.calendarRange(di,di,'DAY_OF_MONTH'))
#                 .first())

In [ ]:
# import dmpipeline.GEE_Processing.GEE_preprocessing_functions as geeprep

# # Calcul indices jour=di
# indices_preproc, QA_dict  = geeprep.preprocessingMODIS(lst_aqua, lst_terra, reflect, landmask, new_collection, QA_dict, CONFIG['CRS_OUT_modis'], GRID_OUT, SALE_OUT, path2key, int(CONFIG['ASSET_EXPORT']))

In [ ]:
# bands = indices_preproc.bandNames().getInfo()
# LST = indices_preproc.select('LST')
# NDWI = indices_preproc.select('NDWI')
# crs_lst = LST.projection().getInfo()['crs']
# crs_ndwi = NDWI.projection().getInfo()['crs']
# QA_di = QA_dict[0].getInfo()['properties']

# print(f'\nbands Di = {bands}')
# print(f'\ncrs lst Di = {crs_lst}')
# print(f'\ncrs ndwi Di = {crs_ndwi}')
# print(f'\nQA_dict Di = {QA_di}')


### Calcul indices MODIS jour Di+1

In [ ]:
# # Extrait produits jour=di+1
# dip1 = 2

# lst_aqua = (LSTaqua_month
#                 .filter(ee.Filter.calendarRange(dip1,dip1,'DAY_OF_MONTH'))
#                 .first())
# lst_terra = (LSTterra_month
#                 .filter(ee.Filter.calendarRange(dip1,dip1,'DAY_OF_MONTH'))
#                 .first())
# reflect = (REFLECTterra_month
#                 .filter(ee.Filter.calendarRange(dip1,dip1,'DAY_OF_MONTH'))
#                 .first())

In [ ]:
# # Calcul indices jour=di+1
# indices_preproc_dip1, QA_dict  = geeprep.preprocessingMODIS(lst_aqua, lst_terra, reflect, landmask, new_collection, QA_dict, CONFIG['CRS_OUT_modis'], GRID_OUT, SALE_OUT, path2key, int(CONFIG['ASSET_EXPORT']))

In [ ]:
# bands = indices_preproc_dip1.bandNames().getInfo()
# LST = indices_preproc_dip1.select('LST')
# NDWI = indices_preproc_dip1.select('NDWI')
# crs_lst = LST.projection().getInfo()['crs']
# crs_ndwi = NDWI.projection().getInfo()['crs']
# QA_dip1 = QA_dict[1].getInfo()['properties']

# print(f'\nbands Di+1 = {bands}')
# print(f'\ncrs lst Di+1 = {crs_lst}')
# print(f'\ncrs ndwi Di+1 = {crs_ndwi}')
# print(f'\nQA_dict Di+1 = {QA_dip1}')

### Test Composition temporelle indices MODIS sur deux dates (Di et Di+1)

In [ ]:
# # Load already processed indices (to avoid running again above lines)
# import ee
# import GEE_Processing.GEE_generic_functions as geegen

# project_id = geegen.googleAuthentification(os.path.join(os.getcwd(),'GEE_Processing'))
# new_collection = 'projects/eo4dm-367504/assets/PREPROC_GLOBAL_INDICES_200301_200301_New_Caledonia_Fr/MODIS'

# di = 1
# indices_preproc = ee.ImageCollection(new_collection).filter(ee.Filter.calendarRange(di,di,'DAY_OF_MONTH')).first()

# dip1 = 2
# indices_preproc = ee.ImageCollection(new_collection).filter(ee.Filter.calendarRange(dip1,dip1,'DAY_OF_MONTH')).first()

# GRID_OUT =  ee.Geometry.Polygon([[[BBOX_OUT['lon_min'], BBOX_OUT['lat_max']],
#                                   [BBOX_OUT['lon_min'], BBOX_OUT['lat_min']],
#                                   [BBOX_OUT['lon_max'], BBOX_OUT['lat_min']],
#                                   [BBOX_OUT['lon_max'], BBOX_OUT['lat_max']]]],None, False)

# SCALE_REFLECT = new_ndwi_di.select('NDWI').projection().nominalScale()

# print(indices_preproc.bandNames().getInfo())
# print(indices_preproc.date().format('YYYYMMdd').getInfo())
# print(indices_preproc_dip1.bandNames().getInfo())
# print(indices_preproc_dip1.date().format('YYYYMMdd').getInfo())


In [ ]:
# # Concatenates both images into a single image collection
# indices_comp = ee.ImageCollection([indices_preproc, indices_preproc_dip1])

In [ ]:
# import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp

# # Computes composite on each band
# INDICES_comp = geecomp.extractComposite(indices_comp, GRID_OUT)

In [ ]:
# LST_comp = INDICES_comp['LST']
# NDWI_comp = INDICES_comp['NDWI']

# band_lst_comp = LST_comp.bandNames().getInfo()
# band_ndwi_comp = NDWI_comp.bandNames().getInfo()
# crs_lst_comp = LST_comp.projection().getInfo()['crs']
# crs_ndwi_comp = NDWI_comp.projection().getInfo()['crs']

# print(f'\nbands comp LST = {band_lst_comp}')
# print(f'\nbands comp NDWI = {band_ndwi_comp}')

# print(f'\ncrs comp LST = {crs_lst_comp}')
# print(f'\ncrs comp NDWI = {crs_ndwi_comp}')

In [ ]:
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# comp_lst_filename = f'MODIS_LST_COMP'
# comp_ndwi_filename = f'MODIS_NDWI_COMP'

In [ ]:
# geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['LST'], comp_lst_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=CONFIG['CRS_OUT_modis'], data_scale=SCALE_REFLECT, data_region=GRID_OUT)
# geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDWI'], comp_ndwi_filename, export_folder=OUTDIR_PATHS[1] path2key=path2key, data_crs=CONFIG['CRS_OUT_modis'], data_scale=SCALE_REFLECT, data_region=GRID_OUT)

### Composition temporelle des indices MODIS par décade (sur un mois)

In [ ]:
# import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp

# # Calcul composites décades
# (LSTcomp_col, NDWIcomp_col, QA_dict, COMP_TYPES,
#  COMP_NANSCORES, NBLST_DATES, NBREFLECT_DATES) = geecomp.processCompositeDecade_MODIS(LSTaqua_month, LSTterra_month, REFLECTterra_month,
#                                                                                       landmask, new_collection, QA_dict, CONFIG['CRS_OUT_modis'], GRID_OUT,
#                                                                                       SALE_OUT, path2key, CONFIG['ASSET_EXPORT'])

In [ ]:
# import time
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# # Exporte composites
# month2find = '{}{:02d}'.format(y,m)
# N_dec = LSTcomp_col.size().getInfo()
# LSTcomp_list = LSTcomp_col.toList(N_dec)
# NDWIcomp_list = NDWIcomp_col.toList(N_dec)


In [ ]:

# for d in range(N_dec):

#     LST_comp = ee.Image(LSTcomp_list.get(d))
#     NDWI_comp = ee.Image(NDWIcomp_list.get(d))
    
#     comp_lst_filename = f'MODIS_LST_{month2find}_{COMP_TYPES[d]}'
#     comp_ndwi_filename = f'MODIS_NDWI_{month2find}_{COMP_TYPES[d]}'

#     start_time = time.time()
#     geegen.exportImage(CONFIG['DRIVE_FOLDER'], LST_comp, comp_lst_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=CONFIG['CRS_OUT_modis'], data_scale=SCALE_REFLECT, data_region=GRID_OUT)
#     geegen.exportImage(CONFIG['DRIVE_FOLDER'], NDWI_comp, comp_ndwi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=CONFIG['CRS_OUT_modis'], data_scale=SCALE_REFLECT, data_region=GRID_OUT)
#     elapsed_time = round(time.time() - start_time)

#     COMP_dict += [ee.Feature(None, {'DATE': month2find,
#                                     'COMPOSITE': COMP_TYPES[d],
#                                     'AquaTerra LST': NBLST_DATES[d],
#                                     'Terra REFLECT': NBREFLECT_DATES[d],
#                                     'COMPOSITE NAN SCORE LST': COMP_NANSCORES[d]['LST'],
#                                     'COMPOSITE NAN SCORE NDWI': COMP_NANSCORES[d]['NDWI'],
    #                                 'COMPOSITE TIME (sec)': elapsed_time})]
    # del start_time, elapsed_time

In [ ]:
# # --- SAVE TABLES (DATA FRAMES) INTO CSV FILES (LOCAL MACHINE) ---
# geegen.exportDataFrame(CONFIG['DRIVE_FOLDER'], QA_dict, QAtable_filename, QAtable_columns, OUTDIR_PATHS[3], path2key)
# geegen.exportDataFrame(CONFIG['DRIVE_FOLDER'], COMP_dict, COMPtable_filename, COMPtable_columns, OUTDIR_PATHS[3], path2key)

### Test complet avec fonction Run MODIS (intègre différents traitements précédents)

In [ ]:
geemain.run_GEEProcessing_MODIS(CONFIG, OUTDIR_PATHS, COLLECTIONS, PARAM, DICT, go_month)